<a href="https://colab.research.google.com/github/NickSlm/ml1/blob/main/Sentiment_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
import tensorflow_datasets as tfds

2023-12-14 11:58:34.726020: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2023-12-14 11:58:34.765253: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-12-14 11:58:34.765287: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-12-14 11:58:34.766014: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-12-14 11:58:34.771425: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2023-12-14 11:58:34.772227: I tensorflow/core/platform/cpu_feature_guard.cc:1

In [2]:
dataset = keras.datasets.imdb.load_data()

17464789/17464789 [==============================] - 2s 0us/step


In [3]:
(X_train, y_train), (X_test, y_test) = dataset

In [4]:
test_seq = X_train[0][:10]

In [5]:
word_index = keras.datasets.imdb.get_word_index()

1641221/1641221 [==============================] - 1s 1us/step


In [6]:
id_to_word = dict((id_ + 3,word) for word, id_ in word_index.items())
id_to_word [1] = "[START]"
id_to_word [2] = "[OOV]"
" ".join(id_to_word[id_] for id_ in test_seq)

'[START] this film was just brilliant casting location scenery story'

In [7]:
dataset, info = tfds.load("imdb_reviews", as_supervised=True, with_info=True)

2023-12-14 11:58:42.287501: W external/local_tsl/tsl/platform/cloud/google_auth_provider.cc:184] All attempts to get a Google authentication bearer token failed, returning an empty token. Retrieving token from files failed with "NOT_FOUND: Could not locate the credentials file.". Retrieving token from GCE failed with "FAILED_PRECONDITION: Error executing an HTTP request: libcurl code 6 meaning 'Couldn't resolve host name', error details: Could not resolve host: metadata.google.internal".


Dl Completed...: 0 url [00:00, ? url/s]
Generating train examples...:  98%|█████████████████████████████████████▏| 24454/25000 [00:04<00:00, 7863.43 examples/s]
                                                                                                                        
Shuffling /home/nick/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteLV4JZI/imdb_reviews-train.tfrecord*...:
Generating test examples...:  99%|██████████████████████████████████████▌| 24689/25000 [00:03<00:00, 7929.69 examples/s]
                                                                                                                        
Shuffling /home/nick/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteLV4JZI/imdb_reviews-test.tfrecord*...: 
Generating unsupervised examples...:  99%|██████████████████████████████▋| 49591/50000 [00:09<00:00, 7994.37 examples/s]
                                                                                                                 

Dataset imdb_reviews downloaded and prepared to /home/nick/tensorflow_datasets/imdb_reviews/plain_text/1.0.0. Subsequent calls will reuse this data.


In [8]:
train_size = info.splits["train"].num_examples
test_size = info.splits["test"].num_examples

In [9]:
def preprocess(X_batch, y_batch):
  X_batch = tf.strings.substr(X_batch, 0, 300)
  X_batch = tf.strings.regex_replace(X_batch, rb"<br\s*/?>", b" ")    # replace all <br>, <br/>
  X_batch = tf.strings.regex_replace(X_batch, b"[^a-zA-Z']", b" ")
  X_batch = tf.strings.split(X_batch)
  return X_batch.to_tensor(default_value=b"<pad>"), y_batch

In [10]:
from collections import Counter

In [11]:
vocab = Counter()
for X_batch, y_batch in dataset["train"].batch(32).map(preprocess):
  for review in X_batch:
    vocab.update(list(review.numpy()))

In [12]:
n_words = 10000
truncated_vocab = [word for word, times in vocab.most_common()[:n_words]]

In [13]:
word_ids = tf.range(len(truncated_vocab), dtype=tf.int64)
words = tf.constant(truncated_vocab)
table_init = tf.lookup.KeyValueTensorInitializer(words, word_ids)
oov_buckets = 1000
table = tf.lookup.StaticVocabularyTable(table_init, oov_buckets)

In [14]:
table.lookup(tf.constant([b"This movie was faaaaaantastic".split()]))

<tf.Tensor: shape=(1, 4), dtype=int64, numpy=array([[   22,    12,    11, 10053]])>

In [15]:
def encode_words(X_batch, y_batch):
  return table.lookup(X_batch), y_batch

train_set = dataset["train"].batch(32).map(preprocess)
train_set = train_set.map(encode_words).prefetch(1)

In [16]:
model = keras.models.Sequential([
    keras.layers.Embedding(n_words + oov_buckets, 128, mask_zero = True, input_shape=[None]),
    keras.layers.GRU(128, return_sequences=True),
    keras.layers.GRU(128),
    keras.layers.Dense(1, activation="sigmoid")
])

Reusing Pretrained Embedding

In [17]:
import tensorflow_hub as hub

In [18]:
model = keras.models.Sequential([
    hub.KerasLayer("https://tfhub.dev/google/tf2-preview/nnlm-en-dim50/1",
                   dtype=tf.string, input_shape=[], output_shape=[50]),
    keras.layers.Dense(128, activation="relu"),
    keras.layers.Dense(1, activation="sigmoid")
])
model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [19]:
datasets, info = tfds.load("imdb_reviews", as_supervised=True, with_info=True)
train_size = info.splits["train"].num_examples
batch_size = 32
train_set = datasets["train"].batch(batch_size).prefetch(1)
print(train_set)

<_PrefetchDataset element_spec=(TensorSpec(shape=(None,), dtype=tf.string, name=None), TensorSpec(shape=(None,), dtype=tf.int64, name=None))>


In [ ]:
history = model.fit(train_set, epochs=5)

Epoch 1/5
782/782 [==============================] - 3s 3ms/step - loss: 0.5485 - accuracy: 0.7240
Epoch 2/5
782/782 [==============================] - 2s 3ms/step - loss: 0.5138 - accuracy: 0.7474
Epoch 3/5
782/782 [==============================] - 2s 2ms/step - loss: 0.5081 - accuracy: 0.7525
Epoch 4/5
777/782 [============================>.] - ETA: 0s - loss: 0.5036 - accuracy: 0.7563